<a href="https://colab.research.google.com/github/planctao/data-flower/blob/main/Module2/02a_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)

# Module 2: PyTorch tensors and automatic differentiation

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=103)

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np

In [2]:
torch.__version__

'2.6.0+cu124'

Tensors are used to encode the signal to process, but also the internal states and parameters of models.

**Manipulating data through this constrained structure allows to use CPUs and GPUs at peak performance.**

Construct a 3x5 matrix, uninitialized:

In [3]:
x = torch.empty(3,5)
print(x.dtype)
print(x)

torch.float32
tensor([[1.5751e-42, 4.5083e-41, 3.4837e-08, 0.0000e+00, 4.2039e-45],
        [4.5083e-41, 3.4836e-08, 0.0000e+00, 1.5765e-42, 4.5083e-41],
        [3.4837e-08, 0.0000e+00, 4.2039e-45, 4.5083e-41, 3.4836e-08]])


If you got an error this [stackoverflow link](https://stackoverflow.com/questions/50617917/overflow-when-unpacking-long-pytorch) might be useful...

In [4]:
x = torch.randn(3,5)
print(x)

tensor([[ 0.2873,  1.2910,  0.7646, -1.0735, -1.1825],
        [ 1.0642,  0.4076,  1.4252, -0.7591, -0.2548],
        [-1.4869, -1.2831, -0.3867,  1.1389, -0.0252]])


In [5]:
print(x.size())

torch.Size([3, 5])


torch.Size is in fact a [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences), so it supports the same operations.

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=272)

In [6]:
x.size()[1]

5

In [7]:
x.size() == (3,5)

True

### Bridge to numpy

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=325)

In [8]:
y = x.numpy()
print(y)

[[ 0.2872529   1.29102     0.7645851  -1.0735136  -1.1824627 ]
 [ 1.0642257   0.40755174  1.4252014  -0.75912076 -0.25478882]
 [-1.4868985  -1.2831483  -0.38667578  1.1388899  -0.02517692]]


In [9]:
a = np.ones(5)
b = torch.from_numpy(a)
print(a.dtype)
print(b)

float64
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [10]:
c = b.long()
print(c.dtype, c)
print(b.dtype, b)

torch.int64 tensor([1, 1, 1, 1, 1])
torch.float64 tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [11]:
xr = torch.randn(3, 5)
print(xr.dtype, xr)

torch.float32 tensor([[-1.0726, -0.2109, -1.0384,  0.4995, -0.3916],
        [-0.1350,  0.7771, -1.2271, -0.0200, -2.9418],
        [ 0.7888, -0.4320,  0.6463,  0.0055,  0.0482]])


In [13]:
resb = xr + b
print(b)
resb

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


tensor([[-0.0726,  0.7891, -0.0384,  1.4995,  0.6084],
        [ 0.8650,  1.7771, -0.2271,  0.9800, -1.9418],
        [ 1.7888,  0.5680,  1.6463,  1.0055,  1.0482]], dtype=torch.float64)

In [14]:
resc = xr + c
resc

tensor([[-0.0726,  0.7891, -0.0384,  1.4995,  0.6084],
        [ 0.8650,  1.7771, -0.2271,  0.9800, -1.9418],
        [ 1.7888,  0.5680,  1.6463,  1.0055,  1.0482]])

Be careful with types!

In [15]:
resb == resc

tensor([[ True, False,  True, False, False],
        [ True, False,  True, False,  True],
        [False,  True, False, False, False]])

In [17]:
torch.set_printoptions(precision=10)

In [18]:
resb[0,1]

tensor(0.7890603244, dtype=torch.float64)

In [19]:
resc[0,1]

tensor(0.7890603542)

In [20]:
resc[0,1].dtype

torch.float32

In [21]:
xr[0,1]

tensor(-0.2109396756)

In [22]:
torch.set_printoptions(precision=4)

### [Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=670)

Broadcasting automagically expands dimensions by replicating coefficients, when it is necessary to perform operations.

1. If one of the tensors has fewer dimensions than the other, it is reshaped by adding as many dimensions of size 1 as necessary in the front; then
2. for every mismatch, if one of the two tensor is of size one, it is expanded along this axis by replicating  coefficients.

If there is a tensor size mismatch for one of the dimension and neither of them is one, the operation fails.

In [23]:
A = torch.tensor([[1.], [2.], [3.], [4.]])
print(A.size())
B = torch.tensor([[5., -5., 5., -5., 5.]])
print(B.size())
C = A + B

torch.Size([4, 1])
torch.Size([1, 5])


In [24]:
C

tensor([[ 6., -4.,  6., -4.,  6.],
        [ 7., -3.,  7., -3.,  7.],
        [ 8., -2.,  8., -2.,  8.],
        [ 9., -1.,  9., -1.,  9.]])

The original (column-)vector
\begin{eqnarray*}
A = \left( \begin{array}{c}
1\\
2\\
3\\
4\\
\end{array}\right)
\end{eqnarray*}
is transformed into the matrix
\begin{eqnarray*}
A = \left( \begin{array}{ccccc}
1&1&1&1&1\\
2&2&2&2&2\\
3&3&3&3&3\\
4&4&4&4&4
\end{array}\right)
\end{eqnarray*}
and the original (row-)vector
\begin{eqnarray*}
B = (5,-5,5,-5,5)
\end{eqnarray*}
is transformed into the matrix
\begin{eqnarray*}
B = \left( \begin{array}{ccccc}
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5
\end{array}\right)
\end{eqnarray*}
so that summing these matrices gives:
\begin{eqnarray*}
A+B = \left( \begin{array}{ccccc}
6&-4&6&-4&6\\
7&-3&7&-3&7\\
8&-2&8&-2&8\\
9&-1&9&-1&9
\end{array}\right)
\end{eqnarray*}

### In-place modification

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=875)

In [25]:
x

tensor([[ 0.2873,  1.2910,  0.7646, -1.0735, -1.1825],
        [ 1.0642,  0.4076,  1.4252, -0.7591, -0.2548],
        [-1.4869, -1.2831, -0.3867,  1.1389, -0.0252]])

In [26]:
xr

tensor([[-1.0726, -0.2109, -1.0384,  0.4995, -0.3916],
        [-0.1350,  0.7771, -1.2271, -0.0200, -2.9418],
        [ 0.7888, -0.4320,  0.6463,  0.0055,  0.0482]])

In [27]:
print(x+xr)

tensor([[-0.7853,  1.0801, -0.2738, -0.5740, -1.5741],
        [ 0.9292,  1.1846,  0.1981, -0.7791, -3.1966],
        [-0.6981, -1.7152,  0.2596,  1.1444,  0.0230]])


In [28]:
x.add_(xr)
print(x)

tensor([[-0.7853,  1.0801, -0.2738, -0.5740, -1.5741],
        [ 0.9292,  1.1846,  0.1981, -0.7791, -3.1966],
        [-0.6981, -1.7152,  0.2596,  1.1444,  0.0230]])


Any operation that mutates a tensor in-place is post-fixed with an ```_```

For example: ```x.fill_(y)```, ```x.t_()```, will change ```x```.

In [29]:
print(x.t())

tensor([[-0.7853,  0.9292, -0.6981],
        [ 1.0801,  1.1846, -1.7152],
        [-0.2738,  0.1981,  0.2596],
        [-0.5740, -0.7791,  1.1444],
        [-1.5741, -3.1966,  0.0230]])


In [30]:
x.t_()
print(x)

tensor([[-0.7853,  0.9292, -0.6981],
        [ 1.0801,  1.1846, -1.7152],
        [-0.2738,  0.1981,  0.2596],
        [-0.5740, -0.7791,  1.1444],
        [-1.5741, -3.1966,  0.0230]])


### Shared memory

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=990)

Also be careful, changing the torch tensor modify the numpy array and vice-versa...

This is explained in the PyTorch documentation [here](https://pytorch.org/docs/stable/torch.html#torch.from_numpy):
The returned tensor by `torch.from_numpy` and ndarray share the same memory. Modifications to the tensor will be reflected in the ndarray and vice versa.

In [ ]:
a = np.ones(5)
b = torch.from_numpy(a)
print(b)

In [ ]:
a[2] = 0
print(b)

In [ ]:
b[3] = 5
print(a)

### Cuda

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1120)

In [31]:
torch.cuda.is_available()

True

In [33]:
#device = torch.device('cpu')
device = torch.device('cuda') # Uncomment this to run on GPU

In [35]:
x.device

device(type='cpu')

In [36]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z,z.type())
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[ 0.2147,  1.9292,  0.3019],
        [ 2.0801,  2.1846, -0.7152],
        [ 0.7262,  1.1981,  1.2596],
        [ 0.4260,  0.2209,  2.1444],
        [-0.5741, -2.1966,  1.0230]], device='cuda:0') torch.cuda.FloatTensor
tensor([[ 0.2147,  1.9292,  0.3019],
        [ 2.0801,  2.1846, -0.7152],
        [ 0.7262,  1.1981,  1.2596],
        [ 0.4260,  0.2209,  2.1444],
        [-0.5741, -2.1966,  1.0230]], dtype=torch.float64)


In [37]:
x = torch.randn(1)
x = x.to(device)

In [38]:
x.device

device(type='cuda', index=0)

In [39]:
# the following line is only useful if CUDA is available
x = x.data
print(x)
print(x.item())
print(x.cpu().numpy())

tensor([-0.0223], device='cuda:0')
-0.022292090579867363
[-0.02229209]


# Simple interfaces to standard image data-bases

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1354)

An example, the [CIFAR10](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10) dataset.

In [40]:
import torchvision

data_dir = 'content/data'

cifar = torchvision.datasets.CIFAR10(data_dir, train = True, download = True)
cifar.data.shape

100%|██████████| 170M/170M [00:03<00:00, 48.8MB/s]


(50000, 32, 32, 3)

Documentation about the [`permute`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.permute) operation.

In [41]:
x = torch.from_numpy(cifar.data).permute(0,3,1,2).float()
x = x / 255
print(x.type(), x.size(), x.min().item(), x.max().item())

torch.FloatTensor torch.Size([50000, 3, 32, 32]) 0.0 1.0


Documentation about the [`narrow(input, dim, start, length)`](https://pytorch.org/docs/stable/torch.html#torch.narrow) operation.

In [ ]:
# Narrows to the first images, converts to float
x = torch.narrow(x, 0, 0, 48)

In [ ]:
x.shape

In [ ]:
# Showing images
def show(img):
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

show(torchvision.utils.make_grid(x, nrow = 12))

In [ ]:
# Kills the green and blue channels
x.narrow(1, 1, 2).fill_(0)
show(torchvision.utils.make_grid(x, nrow = 12))

# Autograd: automatic differentiation

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=40)

When executing tensor operations, PyTorch can automatically construct on-the-fly the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

To be more concrete, we introduce the following example: we consider parameters $w\in \mathbb{R}$ and $b\in \mathbb{R}$ with the corresponding function:
\begin{eqnarray*}
\ell = \left(\exp(wx+b) - y^* \right)^2
\end{eqnarray*}

Our goal here, will be to compute the following partial derivatives:
\begin{eqnarray*}
\frac{\partial \ell}{\partial w}\mbox{ and, }\frac{\partial \ell}{\partial b}.
\end{eqnarray*}

The reason for doing this will be clear when you will solve the practicals for this lesson!

You can decompose this function as a composition of basic operations. This is call the forward pass on the graph of operations.
![backprop1](https://dataflowr.github.io/notebooks/Module2/img/backprop1.png)

Let say we start with our model in `numpy`:

In [ ]:
w = np.array([0.5])
b = np.array([2])
xx = np.array([0.5])#np.arange(0,1.5,.5)

transform these into `tensor`:

In [ ]:
xx_t = torch.from_numpy(xx)
w_t = torch.from_numpy(w)
b_t = torch.from_numpy(b)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=224)

A `tensor` has a Boolean field `requires_grad`, set to `False` by default, which states if PyTorch should build the graph of operations so that gradients with respect to it can be computed.

In [ ]:
w_t.requires_grad

We want to take derivative with respect to $w$ so we change this value:

In [ ]:
w_t.requires_grad_(True)

We want to do the same thing for $b$ but the following line will produce an error!

In [ ]:
b_t.requires_grad_(True)

Reading the error message should allow you to correct the mistake!

In [ ]:
dtype = torch.float64

In [ ]:
b_t = b_t.type(dtype)

In [ ]:
b_t.requires_grad_(True)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=404)

We now compute the function:

In [ ]:
def fun(x,ystar):
    y = torch.exp(w_t*x+b_t)
    print(y)
    return torch.sum((y-ystar)**2)

ystar_t = torch.randn_like(xx_t)
l_t = fun(xx_t,ystar_t)

In [ ]:
l_t

In [ ]:
l_t.requires_grad

After the computation is finished, i.e. *forward pass*, you can call ```.backward()``` and have all the gradients computed automatically.

In [ ]:
print(w_t.grad)

In [ ]:
l_t.backward()

In [ ]:
print(w_t.grad)
print(b_t.grad)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=545)

Let's try to understand these numbers...

![backprop2](https://dataflowr.github.io/notebooks/Module2/img/backprop2.png)

In [ ]:
yy_t = torch.exp(w_t*xx_t+b_t)
print(torch.sum(2*(yy_t-ystar_t)*yy_t*xx_t))
print(torch.sum(2*(yy_t-ystar_t)*yy_t))

`tensor.backward()` accumulates the gradients in  the `grad` fields  of tensors.

In [ ]:
l_t = fun(xx_t,ystar_t)
l_t.backward()

In [ ]:
print(w_t.grad)
print(b_t.grad)

By default, `backward` deletes the computational graph when it is used so that you will get an error below:

In [ ]:
l_t.backward()

In [ ]:
# Manually zero the gradients
w_t.grad.data.zero_()
b_t.grad.data.zero_()
l_t = fun(xx_t,ystar_t)
l_t.backward(retain_graph=True)
l_t.backward()
print(w_t.grad)
print(b_t.grad)

The gradients must be set to zero manually. Otherwise they will cumulate across several _.backward()_ calls.
This accumulating behavior is desirable in particular to compute the gradient of a loss summed over several “mini-batches,” or the gradient of a sum of losses.

In [ ]:
[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)